In [ ]:
import os
print('CUDA_VISIBLE_DEVICES', os.environ['CUDA_VISIBLE_DEVICES'])
os.environ['CUDA_LAUNCH_BLOCKING'] = '1' # does not affect results
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt
import tabulate
from collections import OrderedDict
import itertools
import pickle

In [ ]:
def measure_time(fn, min_samples=10, max_samples=100, max_time_in_sec=100.0):
    """Measure time to run fn. Returns the elapsed time each run."""
    from time import perf_counter_ns
    t = []
    for i in range(max_samples):
        if sum(t) / 1e9 >= max_time_in_sec and i >= min_samples:
            break
        t.append(-perf_counter_ns())
        try:
            ans = fn()
        except Exception as e:
            print(e)
            return np.array([np.nan])
        t[-1] += perf_counter_ns()
        del ans
    print('.', end='')
    return np.array(t) / 1e9

In [ ]:
# cuda device
o3d_cuda_dev = o3d.core.Device(o3d.core.Device.CUDA,0)

In [ ]:
# collects runtimes for all examples
results = OrderedDict()

In [ ]:
# setup dataset examples
datasets = OrderedDict()

pcd = o3d.t.io.read_point_cloud('/root/code/Open3D/small_tower.ply')
points = queries = pcd.point['points']
datasets['sfm'] = {'points': points, 'queries': queries }

# pcd_ds = o3d.t.io.read_point_cloud('canyon.ply')
# points = pcd_ds.point['points'].to(o3d.core.Dtype.Float32)
# pcd_qs = o3d.t.io.read_point_cloud('fluid_1000.ply')
# queries = pcd_qs.point['points'].to(o3d.core.Dtype.Float32)
# datasets['fluid'] = {'points': points, 'queries': queries }

# random data
points = queries = o3d.core.Tensor.from_numpy(np.random.rand(points.shape[0],3).astype(np.float32))
datasets['random'] = {'points': points, 'queries': queries }

In [ ]:
# setup and search functions
def nns_setup(points):
    nns = o3d.core.nns.NearestNeighborSearch(points)
    status = nns.knn_index()
    if not status:
        raise Excepion('index failed')
    return nns

def nns_search(nns, queries, knn):
    ans = nns.knn_search(queries, knn)
    return ans

class O3DKnn:
    def __init__(self):
        self.nns = None
    
    def setup(self, points):
        self.nns = o3d.core.nns.KnnIndex()
        self.nns.set_tensor_data(points)
        return True
    
    def search(self, queries, knn):
        ans = self.nns.knn_search(queries, knn)
        return ans
    
    def clear(self):
        del self.nns
        
class O3DFaiss(O3DKnn):
    def setup(self, points):
        self.nns = o3d.core.nns.FaissIndex()
        self.nns.set_tensor_data(points)
        return True

methods = [O3DKnn(), O3DFaiss()]
method_names = [m.__class__.__name__ for m in methods]

In [ ]:
for method in methods:
    method_name = method.__class__.__name__
    print(method_name)
    for example_name, example in datasets.items():
        print(example_name)
        points = example['points']
        queries = example['queries']

        for knn, step in itertools.product((1,37,64), (1,10,100)):
            print(knn, step)
            points = example['points']
            queries = example['queries']

            points = points[::step].contiguous().to(o3d_cuda_dev)
            queries = queries[::step].contiguous().to(o3d_cuda_dev)

            example_results = {'k': knn, 'num_points': points.shape[0]}

            ans = measure_time(lambda:method.setup(points))
            example_results['knn_gpu_setup'] = ans

            ans = measure_time(lambda :method.search(queries, knn))
            example_results['knn_gpu_search'] = ans

            method.clear()
            o3d.core.cuda.release_cache()


            results[f'{example_name} n={points.shape[0]} k={knn}'] = example_results

            del points
            del queries
            o3d.core.cuda.release_cache()

    with open(f'{method_name}.pkl', 'wb') as f:
        pickle.dump(results, f)

In [ ]:
results = []
for method_name, method in zip(method_names, methods):
    with open(f"{method_name}.pkl", "rb") as f:
        data = pickle.load(f)
        results.append(data)
# with open('knn.pkl', 'rb') as f:
#     knn = pickle.load(f)
# with open('faiss.pkl', 'rb') as f:
#     faiss = pickle.load(f)

In [ ]:
def print_table():
    headers = [''] + [f'{n}_setup' for n in method_names] + [f'{n}_search' for n in method_names]
    rows = []
    
    for x in results[0]:
        r = [x] + list(map(np.median, [ r[x]['knn_gpu_setup'] for r in results] + [r[x]['knn_gpu_search'] for r in results]))
        rows.append(r)
    
    print(tabulate.tabulate(rows, headers=headers))
print_table()